## Keyword Tagging of Oshima Information Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Oshima Data : 부동산 사건사고 정보 
#### 'Info' 컬럼 데이터 클러스터링을 통한 부동산 관련 사건 및 사고 정보 분석

- 1. info 컬럼 데이터 한국어로 번역 
- 2.  딥러닝을 이용한 클로스터링 시도 
- 3.  직접 태깅 시도 

In [2]:
import pandas as pd

In [5]:
colab_path = '/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing/yunyoung/data'
vscode_path = './data'
open = pd.read_csv(colab_path + '/open_data_add_latlng.csv', index_col=0)
airbnb = pd.read_csv(colab_path+'/airbnb_data_add_latlng.csv', index_col=0)
oshima_data = pd.read_csv(colab_path+'/oshima_data_add_latlng.csv', index_col=0)
oshima_data.head()

,district,address,info,date,full name,latitude,longitude
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044


#### 'info' 컬럼 데이터 확인 
데이터는 일본어로 작성되어 있으며, 다양한 사건, 사고 및 상황에 대한 세부 정보를 제공

In [ ]:
oshima_data['info'].value_counts()[-10:]

二階、階段をあがって二番目の部屋\n生活保護者の孤高死                                                                        1
60代男性が孤高死                                                                                          1
建築完成後入居前、飛び降り自殺                                                                                    1
50代男性が病死                                                                                           1
桐ヶ丘中学校男子生徒飛び降り自殺                                                                                   1
1階 角部屋 南向き1K（洋室6帖、K1帖） 専有面積15.5㎡\n告知事項あり\n生活保護相談（60代まで）                                            1
やよい荘１０１号室 死体発見（死後３週間前後と推定）                                                                         1
賃貸一戸建て\n鉄骨造 2階建（屋上あり） 主要採光面：南向き5DK（ DK8帖、洋室9帖、洋室4帖、和室6畳、和室6畳、和室6畳） 専有面積135.12㎡\n告知事項有\n定期借家（1年）    1
死体発見 ．                                                                                             1
多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅                                                          

### 1. info 컬럼 데이터를 한국어로 번역
GoogleTranslator 이용하여 번역


In [ ]:
!pip install deep-translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.4 MB/s eta 0:00:00


In [ ]:
from Dataset_Processing.Utils.Utility import translate_text




In [ ]:
from deep_translator import GoogleTranslator

translator = GoogleTranslator(source='ja', target='ko')

def translate(x):
    translated_text = translator.translate(x)
    return translated_text

In [ ]:
from tqdm.notebook import tqdm_notebook

tqdm_notebook()

oshima_data['translate'] = ""

for i, row in tqdm_notebook(oshima_data.iterrows(), total=len(oshima_data)):
    oshima_data['translate'][i] = translate(row['info'])

0it [00:00, ?it/s]

  0%|          | 0/4833 [00:00<?, ?it/s]

<ipython-input-7-12f0b8a1dd51>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oshima_data['translate'][i] = translate(row['info'])


In [ ]:
oshima_data.to_csv('translate.csv')

In [2]:
cd /content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing

/content/drive/MyDrive/Google Colab/Airbnb/Dataset_Processing


In [1]:
import pandas as pd

oshima_kr = pd.read_csv('../yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [3]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


In [4]:
df_unique['info'].unique()

array(['死体発見', '飛び降り自殺', '2階サウナロイヤル 死体発見', ..., '自殺&死体発見',
       '多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅',
       '多摩ニュータウン永山団地4-2-22号棟105号室\n特別募集住宅'], dtype=object)

In [9]:
# !git clone https://github.com/uoneway/KoBertSum.git


Cloning into 'KoBertSum'...
remote: Enumerating objects: 908, done.
remote: Counting objects: 100% (118/118), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 908 (delta 108), reused 102 (delta 102), pack-reused 790
Receiving objects: 100% (908/908), 15.97 MiB | 13.02 MiB/s, done.
Resolving deltas: 100% (576/576), done.


### 2. 딥러닝을 통한 클러스터링 시도 
### 2-1 허깅페이스 뉴스 요약 모델 : kobart-summarization
Korean News Summarization Model : https://huggingface.co/gogamza/kobart-summarization

In [15]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.1 MB/s eta 0:00:00


In [16]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import BartForConditionalGeneration


In [19]:
tokenizer = PreTrainedTokenizerFast.from_pretrained('gogamza/kobart-summarization')
model = BartForConditionalGeneration.from_pretrained('gogamza/kobart-summarization')


You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BartTokenizer'. 
The class this function is called from is 'PreTrainedTokenizerFast'.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


#### translate 컬럼 첫번째 값을 이용해 요약 시도  

In [23]:
df_unique['translate'][0]

'시체 발견'

In [24]:
raw_input_ids = tokenizer.encode(df_unique['translate'][0])
input_ids = [tokenizer.bos_token_id] + raw_input_ids + [tokenizer.eos_token_id]


#### 오시마 데이터에 적절한 모델이 아님
긴 문장이 아닌 짧은 문장에 대한 성능은 매우 낮다 

'시체 발견' -> '시체 발견으로 시체 처리장치가 작동되는 시체 발견으로 시체 발견'

In [25]:
summary_ids = model.generate(torch.tensor([input_ids]))
tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens=True)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1254: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


'시체 발견으로 시체 처리장치가 작동되는 시체 발견으로 시체 발견'

### 2-2 Keybert와 kiwi 형태소분석기를 사용하여 키워드추출

In [53]:
!pip install keybert
!pip install kiwipiepy

#### 사용 예제 

In [62]:
from keybert import KeyBERT
from kiwipiepy import Kiwi
from transformers import BertModel

text="""
신림역에 칼을 들고 서 있다'는 등 살인 예고 글을 인터넷 커뮤니티에 올린 20대 남성이 구속 상태로 재판에 넘겨졌습니다. 
서울중앙지검 형사3부(부장검사 김수민)는 오늘(31일), 29살 최 모 씨를 협박죄 및 위계공무집행방해죄 혐의로 구속 기소했습니다. 
지난달 26일, 최 씨는 한 인터넷 커뮤니티 게시판에 '신림역에 칼을 들고 서 있습니다. 이제부터 사람 죽인다'는 글을 게시했습니다. 
관련 112 신고를 받고 약 20명의 경찰관이 현장에 출동하기도 했습니다. 검찰은 최 씨의 게시글로 다수가 불안을 느끼고 경찰력이 낭비된 점을 들어 협박죄와 위계공무집행방해죄를 함께 적용했습니다. 
검찰은 "시민의 일상을 위협하고 치안 공백을 초래하는 동종 범죄에 엄정 대처할 것"이라고 밝혔습니다. 
"""



- keyphrase_ngram_range : 몇개의 ngram으로 사용할것인가.
- top_n : 몇개의 키워드를 뽑을것인가
- stop_words : 불용어 처리를 할것인가

In [56]:
model = BertModel.from_pretrained('skt/kobert-base-v1')
kw_model = KeyBERT(model)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=10)

#### 키워드 : 가중치 형태로 출력

In [63]:
keywords

[('살인', 0.6339),
 ('범죄에', 0.5276),
 ('구속', 0.5231),
 ('혐의로', 0.4217),
 ('협박죄', 0.4147),
 ('검찰은', 0.3816),
 ('경찰력이', 0.3633),
 ('20대', 0.3527),
 ('20명의', 0.3482),
 ('칼을', 0.3416)]

In [64]:
kiwi = Kiwi()
kiwi.analyze(text)

[([Token(form='신림', tag='NNP', start=1, len=2),
   Token(form='역', tag='NNG', start=3, len=1),
   Token(form='에', tag='JKB', start=4, len=1),
   Token(form='칼', tag='NNG', start=6, len=1),
   Token(form='을', tag='JKO', start=7, len=1),
   Token(form='들', tag='VV', start=9, len=1),
   Token(form='고', tag='EC', start=10, len=1),
   Token(form='서', tag='VV', start=12, len=1),
   Token(form='어', tag='EC', start=12, len=1),
   Token(form='있', tag='VX', start=14, len=1),
   Token(form='다', tag='EF', start=15, len=1),
   Token(form="'", tag='SSC', start=16, len=1),
   Token(form='는', tag='ETM', start=17, len=1),
   Token(form='등', tag='NNB', start=19, len=1),
   Token(form='살인', tag='NNG', start=21, len=2),
   Token(form='예고', tag='NNG', start=24, len=2),
   Token(form='글', tag='NNG', start=27, len=1),
   Token(form='을', tag='JKO', start=28, len=1),
   Token(form='인터넷', tag='NNG', start=30, len=3),
   Token(form='커뮤니티', tag='NNG', start=34, len=4),
   Token(form='에', tag='JKB', start=38, len=

#### 키워드를 뽑기위해 명사만 추출하는 함수 생성

In [65]:
def noun_extractor(text):
    results = []
    result = kiwi.analyze(text)
    for token, pos, _, _ in result[0][0]:
        if len(token) != 1 and pos.startswith('N'):
            results.append(token)
    return results

In [66]:
nouns = noun_extractor(text)
nouns

['신림',
 '살인',
 '예고',
 '인터넷',
 '커뮤니티',
 '남성',
 '구속',
 '상태',
 '재판',
 '서울중앙지검',
 '형사',
 '부장',
 '검사',
 '김수미',
 '오늘',
 '협박',
 '위계',
 '공무',
 '집행',
 '방해죄',
 '혐의',
 '구속',
 '기소',
 '지난달',
 '인터넷',
 '커뮤니티',
 '게시판',
 '신림역',
 '이제',
 '사람',
 '게시',
 '관련',
 '신고',
 '경찰관',
 '현장',
 '출동',
 '검찰',
 '게시',
 '다수',
 '불안',
 '경찰력',
 '낭비',
 '협박',
 '위계',
 '공무',
 '집행',
 '방해',
 '적용',
 '검찰',
 '시민',
 '일상',
 '위협',
 '치안',
 '공백',
 '초래',
 '동종',
 '범죄',
 '엄정',
 '대처']

In [68]:
text = ' '.join(nouns)
text

'신림 살인 예고 인터넷 커뮤니티 남성 구속 상태 재판 서울중앙지검 형사 부장 검사 김수미 오늘 협박 위계 공무 집행 방해죄 혐의 구속 기소 지난달 인터넷 커뮤니티 게시판 신림역 이제 사람 게시 관련 신고 경찰관 현장 출동 검찰 게시 다수 불안 경찰력 낭비 협박 위계 공무 집행 방해 적용 검찰 시민 일상 위협 치안 공백 초래 동종 범죄 엄정 대처'

#### 키워드 별 가중치 확인 

In [70]:
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('구속', 0.6219),
 ('범죄', 0.5988),
 ('경찰력', 0.548),
 ('형사', 0.5304),
 ('경찰관', 0.5191),
 ('살인', 0.4972),
 ('협박', 0.4838),
 ('검찰', 0.4786),
 ('위협', 0.461),
 ('기소', 0.4571),
 ('혐의', 0.4412),
 ('집행', 0.4241),
 ('방해죄', 0.3246),
 ('게시', 0.3237),
 ('서울중앙지검', 0.3182),
 ('재판', 0.3019),
 ('인터넷', 0.2997),
 ('커뮤니티', 0.2836),
 ('시민', 0.2829),
 ('예고', 0.2749)]

### oshimadata - Keybert 적용하여 키워드 추출 시도 

In [74]:
df_unique['translate'].tail(20)

3624                            사쿠라카미미즈 빌레지\n2층 방번 불명 고고사
3625                                           고지사항 있음 물건
3626                            405호실 목욕탕에서 자연사\n발견 2개월 후
3627    아파트 1층의 1번 안쪽 방에서 젊은 남성이 목을 매달아 자살.\n현재는 한 집에 ...
3628       3층\n혼자 사는 40대 여성이 고고사\n고양이가 몇 마리 감겨진 기억이 있습니다.
3629                                                   자살
3630                                            뛰어 내리는 자살
3631    401호실 병사(사건성 없음)\n\n정기계약(3년간) 집세 할인(14만정)\n이후는...
3632                             프레일 아파트 아시카 공원 604호실 고지유
3633                                             심리적 하자 유
3634                                     102호실. 고지 사항 있음.
3635                                                시체 발견
3636                                      201호실 심리적 하자 있음
3637                                                  고고사
3638                                  붉은 루테시아 차 안에서 구운 자살
3639                                            심리적 하자 있음
3640    "심리적 하자 있음"이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시...
3641          

#### 3627 번쨰 컬럼을 이용해 키워드 추출 적용 

In [76]:
print(df_unique['translate'][3627])
nouns = noun_extractor(df_unique['translate'][3627])
nouns

아파트 1층의 1번 안쪽 방에서 젊은 남성이 목을 매달아 자살.
현재는 한 집에 재건축이 끝나고 있다.


['아파트', '안쪽', '남성', '자살', '현재', '건축']

In [77]:
text = ' '.join(nouns)
keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=20)
keywords

[('안쪽', 0.6878),
 ('자살', 0.6009),
 ('아파트', 0.4692),
 ('남성', 0.412),
 ('건축', 0.4044),
 ('현재', 0.3832)]

#### 전체 translate 컬럼에 시도 

#### 명사 추출  

In [78]:
df_unique['keyword'] = df_unique['translate'].map(noun_extractor)
df_unique

,district,address,info,date,full name,latitude,longitude,translate,keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살]
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]"
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살]
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살]
...,...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음,"[심리, 하자]"
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시...","[심리, 하자, 설명, 에이블, 카미이시, 카미이점]"
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견,"[자살, 시체, 발견]"
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택,"[타마, 뉴타운, 영산, 단지, 호실, 특별, 모집, 주택]"


#### 키워드별 가중치 추출 

In [79]:
def get_topN_keyword(keyword):
  text = ' '.join(keyword)
  keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words=None, top_n=5)
  return keywords


In [81]:
df_unique['Top 5 keyword'] = df_unique['keyword'].map(get_topN_keyword)
df_unique.head(20)

,district,address,info,date,full name,latitude,longitude,translate,keyword,Top 5 keyword
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,"[시체, 발견]","[(발견, 0.8794), (시체, 0.8008)]"
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,"[사우나, 로얄, 시체, 발견]","[(사우나, 0.9334), (로얄, 0.7064), (발견, 0.6721), (시..."
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,[자살],"[(자살, 1.0)]"
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,[자살],"[(자살, 1.0)]"
5,新宿区,東京都新宿区西新宿七丁目5-25,20階\n心理的瑕疵あり,不明,東京都 新宿区 西新宿 七丁目 5番 25号,35.697594,139.698837,20층\n심리적 하자 있음,"[심리, 하자]","[(하자, 0.8549), (심리, 0.8209)]"
6,新宿区,東京都新宿区歌舞伎町二丁目35-6,コインロッカー内に乳児の腐乱死体。,平成30年6月12日,東京都 新宿区 歌舞伎町 二丁目 35番 6号,35.696503,139.702469,동전 사물함에 유아의 부란 시체.,"[동전, 사물함, 유아, 시체]","[(유아, 0.881), (시체, 0.5597), (사물함, 0.4752), (동전..."
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,902호실에서 자살 있음.,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,이세탄 테넌트 관계자의 목 매달아 자살,"[이세탄, 테넌트, 관계자, 자살]","[(자살, 0.7929), (이세탄, 0.6237), (테넌트, 0.5543), (..."
9,新宿区,東京都新宿区新宿五丁目18-20 ビジネスホテルミラベル,309号室 首吊り自殺,平成20年12月4日,東京都 新宿区 新宿 五丁目 18番 20号,35.693932,139.706482,309호실 목 매달아 자살,"[호실, 자살]","[(자살, 0.9446), (호실, 0.5454)]"


### 정리 

KeyBERT 키워드 추출 도구와 Kiwi 형태소 분석기를 사용하여 키워드를 추출하려고 시도했지만, 원하는 키워드를 성공적으로 추출하지 못했습니다.

- "2층 사우나 로얄 시체 발견" -> [(사우나, 0.9334), (로얄, 0.7064), (발견, 0.6721)]

- "5층에서 뛰어내리다" -> []

자살, 타살, 추락사와 같은 사건과 사고를 구별되는 카테고리로 생성하려는 시도는 원하는 결과를 얻지 못했기 때문에, 직접 태깅하는 것이 가장 적절한 접근 방식으로 판단되었습니다.



### 0828 태깅

In [2]:
import pandas as pd

oshima_kr = pd.read_csv('../yunyoung/data/translate.csv', index_col=0)
oshima_kr

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
4844,多摩市,東京都八王子市大塚640-13ハウスオブパレ,隣室で謎の死。当時私が住んでいた307号室に謎の黒い影が頻繁に出て来るようになりました。,2006年4月頃,東京都 八王子市 大塚 640番地,35.637493,139.422913,이웃실에서 수수께끼의 죽음. 당시 제가 살았던 307호실에 수수께끼의 검은 그림자가...
4845,多摩市,東京都八王子市大塚769-8,代表取締役が恋人のゲイと喧嘩し、刺殺事故,2015年頃,東京都 八王子市 大塚 769番地,35.638660,139.424469,대표 이사가 연인의 게이와 싸우고 자살 사고
4846,多摩市,東京都多摩市和田2009-1,飛び降り自殺,2018年2月,東京都 多摩市 和田 2009番地,35.639858,139.427063,뛰어 내리는 자살
4847,多摩市,東京都八王子市大塚,1993年ごろかな。10号館あたりで学生が飛び降り自殺をした。何階からかは忘れたけど、渡り廊...,平成24年10月1日,東京都 八王子市 大塚,35.639523,139.418701,1993년경일까. 10호관 근처에서 학생이 뛰어내려 자살을 했다. 몇 층부터는 잊었...


#### 주소 기준 중복값 제거

In [6]:
oshima_kr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4833 entries, 0 to 4848
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   4833 non-null   object 
 1   address    4833 non-null   object 
 2   info       4833 non-null   object 
 3   date       4833 non-null   object 
 4   full name  4833 non-null   object 
 5   latitude   4833 non-null   float64
 6   longitude  4833 non-null   float64
 7   translate  4833 non-null   object 
dtypes: float64(2), object(6)
memory usage: 339.8+ KB


In [7]:
df_unique = oshima_kr.drop_duplicates(subset=['address'])
df_unique.reset_index(drop=True, inplace = True)
df_unique

,district,address,info,date,full name,latitude,longitude,translate
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시..."
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택


#### 번역 확인

In [16]:
df_info_suicide = df_unique.loc[df_unique['translate']=='자살', ['info', 'translate']]
df_info_suicide

,info,translate
12,刺殺,자살
16,刺殺,자살
27,刺殺,자살
39,自殺,자살
53,自殺,자살
...,...,...
3559,自殺,자살
3575,自殺,자살
3603,自殺,자살
3608,刺殺,자살


In [17]:
df_info_suicide['info'].unique()

array(['刺殺', '自殺'], dtype=object)

In [30]:
df_info_suicide2 = df_unique[df_unique['info'].str.contains('刺殺')][['info', 'translate']]
df_info_suicide2

,info,translate
12,刺殺,자살
16,刺殺,자살
27,刺殺,자살
102,刺殺,자살
108,刺殺,자살
...,...,...
3315,刺殺,자살
3408,刺殺,자살
3564,アパートの室内で刺殺。,아파트 실내에서 자살.
3608,刺殺,자살


-  刺殺 는 자살이 아닌 척살(찔러 죽이다) 의미로 주로 쓰임 

In [31]:
df_info_suicide2['translate'].unique()

array(['자살', '구내 자살', '세키 코포 102호실\n누나가 여동생을 자살',
       '2011년 11월 28일\n히가시닛포리 남성직원 자살 사건', '여성 관계 문제로 남성이 자살',
       '현 동전 세탁이 되기 전의 빌딩 시\n남성 1명 자살 현장 터', '2층 남성 자살', '미병자살',
       '남편이 아내를 자살.', '대표 이사가 연인의 게이와 싸우고 자살 사고', '일하는 중국인끼리의 싸움으로 자살',
       '패밀 하이츠 101호실, 치정의 흠집에서 자살 사건이 있던 물건. 실내 혈흔이 날아간 부분만 벽지를 바꿔 놓고 있어 곳곳 벽지가\n다르게 부자연스럽다.',
       '대마초 주의되어 어머니를 자살 사이타마·가와구치, 23세 장남 체포\n\n20일 오전 0시 5분쯤, 사이타마현 가와구치시 아사히 5가의 맨션 「그란데 아사히」 3층의 직장인 모토키 마사오씨(55)쪽으로, 아내의 미츠요씨(48)가 목 등으로부터 피를 흘려 넘어지고 있는 것을 110번으로 달려간 가와구치 서원이 찾아냈다. 동거의 장남이 “대마를 빨고 있는 것을 주의받고 머리에 와 주었다”고 부엌칼로 찔린 것을 인정했기 때문에, 동서는 살인 미수의 현행범으로 체포했다. 무직의 건인 용의자(23)에서, 미쓰요씨는 반송처의 병원에서 사망, 동서는 살인 용의로 전환해 조사하고 있다. 조사에서는 건인 용의자는 19일 오후 11시 50분쯤 광대씨의 목을 부엌칼로 찔렀다. 서원이 달리자 건인 용의자는 지인의 무직남(23)과 방 안에 있었다. 두 사람은 함께 대마초를 빨고 있었다고 하고, 동서는 대마 단속법 위반(소지)의 현행범으로 남자를 체포했다. 이웃 사람이 "싸움을 하는 것 같다"고 110번했다.',
       '구타·자살',
       '이곳은 이전 시닥스 히가시 쿠루메 점이었습니다.\n당시 이웃에서 타키 야마 축제가 열렸습니다.\n시닥스에 내점했지만, 남자 쪽이 여성을 자살했다.\n그 후, 그 방(확실히 2F)에 신선이 설치되어, 모금 소금도 된

In [33]:
df_unique[df_unique['info'].str.contains('刺殺')]

,district,address,info,date,full name,latitude,longitude,translate
12,新宿区,東京都新宿区歌舞伎町一丁目11-5扇園ビル1階,刺殺,平成20年4月8日,東京都 新宿区 歌舞伎町 一丁目 11番 5号,35.695019,139.702698,자살
16,新宿区,東京都新宿区歌舞伎町二丁目3-23新宿セントラルビル3F,刺殺,平成19年4月5日,東京都 新宿区 歌舞伎町 二丁目 3番 23号,35.696857,139.707092,자살
27,新宿区,東京都新宿区歌舞伎町二丁目2-21ライオンズマンション歌舞伎町,刺殺,令和5年5月29日,東京都 新宿区 歌舞伎町 二丁目 2番 21号,35.695518,139.706818,자살
102,中央区,福岡県福岡市中央区大名二丁目6-11 Fukuoka Growth Next 1階トイレ,刺殺,平成30年6月24日,福岡県 福岡市 中央区 大名 二丁目 6番 11号,33.588894,130.395050,자살
108,葛飾区,東京都葛飾区立石五丁目9-11,刺殺,平成20年12月26日,東京都 葛飾区 立石 五丁目 9番 11号,35.743042,139.844772,자살
...,...,...,...,...,...,...,...,...
3315,大田区,神奈川県川崎市川崎区駅前本町24-2サンリバーオオモリ10階西端の部屋,刺殺,令和4年4月27日,神奈川県 川崎市 川崎区 駅前本町 24番,35.534294,139.701263,자살
3408,中野区,東京都中野区弥生町五丁目22-2中野スカイパレス307,刺殺,平成18年12月2日,東京都 中野区 弥生町 五丁目 22番 2号,35.689125,139.666351,자살
3564,豊島区,東京都板橋区幸町20−２,アパートの室内で刺殺。,1974年頃,東京都 板橋区 幸町 20番 2号,35.744095,139.699921,아파트 실내에서 자살.
3608,杉並区,東京都杉並区高円寺南一丁目1-18,刺殺,令和2年4月7日,東京都 杉並区 高円寺 南一丁目 1番 18号,35.699280,139.662125,자살


In [37]:
oshima_fix = df_unique.copy()

oshima_fix.loc[oshima_fix['info'].str.contains('刺殺') & oshima_fix['translate'].str.contains('자살'), 'translate'] = oshima_fix['translate'].str.replace('자살', '칼에 찔려 사망')


In [41]:
oshima_fix[oshima_fix['info'].str.contains('刺殺')].head(10)

,district,address,info,date,full name,latitude,longitude,translate
12,新宿区,東京都新宿区歌舞伎町一丁目11-5扇園ビル1階,刺殺,平成20年4月8日,東京都 新宿区 歌舞伎町 一丁目 11番 5号,35.695019,139.702698,칼에 찔려 사망
16,新宿区,東京都新宿区歌舞伎町二丁目3-23新宿セントラルビル3F,刺殺,平成19年4月5日,東京都 新宿区 歌舞伎町 二丁目 3番 23号,35.696857,139.707092,칼에 찔려 사망
27,新宿区,東京都新宿区歌舞伎町二丁目2-21ライオンズマンション歌舞伎町,刺殺,令和5年5月29日,東京都 新宿区 歌舞伎町 二丁目 2番 21号,35.695518,139.706818,칼에 찔려 사망
102,中央区,福岡県福岡市中央区大名二丁目6-11 Fukuoka Growth Next 1階トイレ,刺殺,平成30年6月24日,福岡県 福岡市 中央区 大名 二丁目 6番 11号,33.588894,130.395050,칼에 찔려 사망
108,葛飾区,東京都葛飾区立石五丁目9-11,刺殺,平成20年12月26日,東京都 葛飾区 立石 五丁目 9番 11号,35.743042,139.844772,칼에 찔려 사망
172,国分寺市,東京都八王子市元本郷町四丁目13-2都営元本郷町四丁目アパート2号棟108,刺殺,平成26年6月13日,東京都 八王子市 元本郷町 四丁目 13番,35.665142,139.313263,칼에 찔려 사망
291,新宿区,東京都新宿区歌舞伎町一丁目115扇園ビル1階,刺殺,平成20年4月8日,東京都 新宿区 歌舞伎町 一丁目,35.695393,139.701294,칼에 찔려 사망
294,新宿区,東京都新宿区歌舞伎町二丁目323新宿セントラルビル3F,刺殺,平成19年4月5日,東京都 新宿区 歌舞伎町 二丁目,35.696556,139.703827,칼에 찔려 사망
306,新宿区,東京都新宿区歌舞伎町二丁目221ライオンズマンション歌舞伎町,刺殺,令和5年5月29日,東京都 新宿区 歌舞伎町 二丁目,35.696556,139.703827,칼에 찔려 사망
344,新宿区,東京都新宿区新宿三丁目191,敷地内 刺殺,平成21年7月19日,東京都 新宿区 新宿 三丁目,35.691227,139.703568,구내 칼에 찔려 사망


In [40]:
oshima_fix[oshima_fix['info'].str.contains('自殺')].head()

,district,address,info,date,full name,latitude,longitude,translate
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살
7,新宿区,東京都新宿区歌舞伎町二丁目8-2,902号室で自殺あり。,2020年7月初旬,東京都 新宿区 歌舞伎町 二丁目 8番 2号,35.694901,139.705887,902호실에서 자살 있음.
8,新宿区,東京都新宿区新宿五丁目16-10,伊勢丹テナント関係者の首吊り自殺,2008年頃,東京都 新宿区 新宿 五丁目 16番 10号,35.693150,139.706360,이세탄 테넌트 관계자의 목 매달아 자살


In [44]:
oshima_fix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   3644 non-null   object 
 1   address    3644 non-null   object 
 2   info       3644 non-null   object 
 3   date       3644 non-null   object 
 4   full name  3644 non-null   object 
 5   latitude   3644 non-null   float64
 6   longitude  3644 non-null   float64
 7   translate  3644 non-null   object 
dtypes: float64(2), object(6)
memory usage: 227.9+ KB


In [42]:
oshima_fix.to_csv('oshima_translate_fix.csv')

#### 직접 태깅하기 

In [43]:
keyword_mapping = {
    
    '火災': '화재',
    '死体': '시체',
    '遺体' : '시체',
    'ミイラ' : '시체',
    '入居者死亡を発見' : '시체',
    'トイレで心肺停止女性発見' : '시체',
    '自殺': '자살',
    '首吊り' : '자살',
    '飛び降り' : '자살',
    '無理心中' : '자살',
    '心中' : '자살',
    '縊死' : '자살',
    '患者死亡' : '자살',
    '孤高死': '고독사',
    '孤独死' : '고독사',
    'て独' : '고독사',
    '餓死' : '고독사',
    '告知事項' : '부동산고지',
    '心理的瑕疵' : '부동산고지',
    '転落死': '추락사',
    'から転落' : '추락사',
    '転落' : '추락사',
    '刺殺': '타살',
    '絞殺' : '타살',
    '射殺' : '타살',
    '車両事故死' : '타살',
    '傷害致死' : '타살',
    '撲殺' : '타살',
    '殺人' : '타살',
    '殴殺' : '타살',
    '虐待' : '타살',
    '遺棄' : '타살',
    '病死' : '질병사',
    '過労死' : '질병사',
    '自然死' : '질병사',
    '熱中症による死亡' : '질병사',
    '業務上過失致死' : '질병사',
    '患者死亡' : '질병사',
    '倒れていて死亡' : '질병사',
    '事故死' : '사고사',
    '傷害のち死亡' : '사고사',
    'エレベータと壁に挟まれ死亡' : '사고사',
    '中毒' : '사고사',
    '死亡事故' : '사고사',
    '溺死' : '익사',
    '水没' : '익사',
    '変死' : '의문사',
    '不審死' : '의문사', 
    '原因不明の事故' : '의문사',
    '突然死' : '의문사',
    '未解決事件' : '의문사',
    '不自然死' : '의문사',
    '不明' : '의문사',
    '精神的瑕疵' : '심신미약',
    '事故物件' : '주택내사고',
    '特別募集' : '주택내사고',
    '事故住宅' : '주택내사고',
    '患者死亡' : '주택내사고',
    '特別募集住宅' : '기타'}

In [50]:
import warnings
import numpy as np 

warnings.filterwarnings("ignore")


def tagging(df):
    df.loc[:, 'crimeType'] = np.nan
    for idx in range(df.shape[0]):
        for key, value in keyword_mapping.items():
            try:
                if key in str(df.loc[idx, 'info']):
                    df.loc[idx, 'crimeType'] = value
                    break
            except KeyError:
                continue

    return df

In [51]:
tagging(oshima_fix)

,district,address,info,date,full name,latitude,longitude,translate,crimeType
0,新宿区,東京都新宿区新宿三丁目22-7指田ビル地下1階,死体発見,平成24年11月5日,東京都 新宿区 新宿 三丁目 22番 7号,35.693264,139.701309,시체 발견,시체
1,新宿区,東京都新宿区西新宿一丁目3-9,飛び降り自殺,令和4年,東京都 新宿区 西新宿 一丁目 3番 9号,35.693218,139.698166,뛰어 내리는 자살,자살
2,新宿区,東京都新宿区歌舞伎町一丁目27-5 中台ビル,2階サウナロイヤル 死体発見,平成20年1月5日,東京都 新宿区 歌舞伎町 一丁目 27番 5号,35.695450,139.700623,2층 사우나 로얄 시체 발견,시체
3,新宿区,東京都新宿区歌舞伎町一丁目20-2アパホテル新宿歌舞伎町タワー↓,飛び降り自殺,令和3年5月11日,東京都 新宿区 歌舞伎町 一丁目 20番 2号,35.695652,139.701401,뛰어 내리는 자살,자살
4,新宿区,東京都新宿区西新宿七丁目5-9プロスペクト・アクス・ザ・タワー新宿2008,首吊り自殺,平成27年9月26日,東京都 新宿区 西新宿 七丁目 5番 9号,35.696812,139.698044,목 매달아 자살,자살
...,...,...,...,...,...,...,...,...,...
3639,杉並区,東京都杉並区善福寺三丁目6-13,心理的瑕疵あり,不明,東京都 杉並区 善福寺 三丁目 6番 13号,35.716324,139.592682,심리적 하자 있음,부동산고지
3640,杉並区,東京都杉並区上井草三丁目37-15プチコ－トＵ201号室,「心理的瑕疵あり」と説明あり (LIFULL HOME'S - エイブル上石神井店),不明,東京都 杉並区 上井草 三丁目 37番 15号,35.723637,139.600052,"""심리적 하자 있음""이라고 설명 있음 (LIFULL HOME'S - 에이블 카미이시...",부동산고지
3641,多摩市,東京都多摩市諏訪四丁目3-3都営諏訪団地4-3-3号棟105,自殺&死体発見,平成24年8月7日,東京都 多摩市 諏訪 四丁目 3番,35.622181,139.458511,자살 & 시체 발견,시체
3642,多摩市,東京都多摩市永山四丁目3-14,多摩ニュータウン永山団地4-3-14号棟102号室\n特別募集住宅,不明,東京都 多摩市 永山 四丁目 3番,35.617443,139.450256,타마 뉴타운 영산단지 4-3-14호동 102호실\n특별 모집 주택,주택내사고


In [84]:
oshima_fix.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3644 entries, 0 to 3643
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   district   3644 non-null   object 
 1   address    3644 non-null   object 
 2   info       3644 non-null   object 
 3   date       3644 non-null   object 
 4   full name  3644 non-null   object 
 5   latitude   3644 non-null   float64
 6   longitude  3644 non-null   float64
 7   translate  3644 non-null   object 
 8   crimeType  3395 non-null   object 
dtypes: float64(2), object(7)
memory usage: 256.3+ KB
